In [1]:
from collections import Counter
import numpy as np
from typing import Dict
from copy import copy
from tqdm import tqdm

# Чтение и предобработка

In [2]:
ru_corpus = ''

for file_name in ['AnnaKarenina.txt', 'WarAndPeace.txt']: 
    with open(file_name, 'r') as f:
        ru_corpus += f.read()
    
with open('WarAndPeaceEng.txt', 'r', encoding='utf-8-sig') as f:
    en_corpus = f.read()

In [3]:
ru_corpus[90:220]

'тавшей афоризмом фразой: «Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему». Это книга о'

In [4]:
ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

In [5]:
np.random.seed(42)

In [6]:
def remove_capitals_punctuations(text: str) -> str:
    output = ''
    for l in text.lower():
        if l in ALPHABET:
            output += ''.join(l)
    return output

In [7]:
ru_corpus = remove_capitals_punctuations(ru_corpus) 

In [8]:
# просто убедиться, что пунктуация и заглавные буквы обработаны
ru_corpus[90:220]

'фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о люб'

# Часть 1 - Базовый частотный метод

In [9]:
def get_unigrams(preprocessed_text: str) -> Dict:
    corpus_counts = Counter(preprocessed_text)
    freqs = {}
    for k, v in corpus_counts.items():
            freqs[k] = v / sum(corpus_counts.values())
    return dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))

In [10]:
def encode_text(corpus_freq_counter: Dict, text: str) -> str:
    initial_mapping = list(corpus_freq_counter.keys())
    mixed_mapping = np.random.permutation(initial_mapping)
    encoder = dict(zip(initial_mapping, mixed_mapping))
    return "".join([encoder[ngram] for ngram in text])


def decode_text(corpus_freq_counter: Dict, encoded_text: str) -> str:
    encoded_text_count = Counter(encoded_text)
    encoded_text_freqs = {}
    for k, v in encoded_text_count.items():
        encoded_text_freqs[k] = v / sum(encoded_text_count.values())
    encoded_text_freqs = dict(sorted(encoded_text_freqs.items(), key=lambda x: x[1], reverse=True))
    decoder = dict(zip(encoded_text_freqs.keys(), corpus_freq_counter.keys()))
    return "".join([decoder[ngram] for ngram in encoded_text])

In [11]:
corpus_unigram_freqs = get_unigrams(ru_corpus)

In [12]:
example = ru_corpus[10400: 10900]

In [13]:
encoded_example = encode_text(corpus_unigram_freqs, example)

In [14]:
decoded_example = decode_text(corpus_unigram_freqs, encoded_example)

## Часть 1 - Результаты

In [15]:
len(example) == len(encoded_example) == len(decoded_example)

True

In [16]:
example

'оторые в первые времена христианства выходили возвещать добрую весть и так горячо верили что умирали за свою верутолстой с большим вниманием относился к этим идеям его герой константин левин тоже размышляет над тем что ему сказал брат николай разговор брата о коммунизме заставил его задуматься он считал переделку экономических условий вздором но он всегда чувствовал несправедливость своего избытка в сравнении с бедностью народа и впоследствии уже накануне  года беседуя с рабочими прохоровской ма'

In [17]:
encoded_example

'ызы щюпвпмю вщюпв юьюлхпш йдзйхлдзвхпвщшыяйъйпвыцвюсхзопяыа итпвюдзопйпзхнпеы кжыпвю йъйпжзыпиьй хъйпцхпдвытпвю изыъдзычпдпаыъогйьпвлйьхлйюьпызлыдйъдкпнпбзйьпйяюкьпюеыпею ычпнылдзхлзйлпъювйлпзыфюп хцьщгъкюзплхяпзюьпжзыпюьипднхцхъпа хзплйныъхчп хцеывы па хзхпыпныььилйцьюпцхдзхвйъпюеыпцхяиьхзодкпылпджйзхъпмю юяюънипбнылыьйжюднйшпидъывйчпвцяы ыьплыпылпвдюеяхпживдзвывхъплюдм хвюяъйвыдзопдвыюеыпйцащзнхпвпд хвлюлййпдпаюялыдзотплх ыяхпйпвмыдъюядзвййпифюплхнхлилюппеыяхпаюдюяикпдп хаыжйьйпм ышы ывднычпьх'

In [18]:
decoded_example

'осовзе и чевизе иверела йвнтсналтсиа изйоднкн иопиецасы доявмш иетсы н сау ьовбго иевнкн гсо мрнвакн па тиош иевмсоктсож т яокыхнр илнралнер ослотнктб у юснр ндебр еьо ьевож уолтсалснл кеинл соэе вапрзхкбес лад сер гсо ерм туапак явас лнуокаж вапьоиов яваса о уоррмлнпре патсаинк еьо падмрасытб ол тгнсак чеведекум юуолорнгетунй мткоинж ипдовор ло ол итеьда гмитсиоиак летчваиедкниотсы тиоеьо нпязсуа и тваилелнн т яедлотсыш лавода н ичоткедтсинн мэе лауалмле  ьода яетедмб т ваяогнрн чвойовоитуож ра'

Получилась полная ерунда, но другого и не ожидалось =)

# Часть 2 - Биграммы

In [19]:
def get_ngrams(text: str, ngram_len: int) -> Dict:
    ngrams = [text[i: i + ngram_len] for i in range(len(text) - ngram_len + 1)]
    ngrams_count = Counter(ngrams)
    ngrams_count_dict = {k[0] + k[1]: v for k, v in ngrams_count.items()}
    freqs = {}
    for k, v in ngrams_count_dict.items():
        freqs[k] = v / sum(ngrams_count_dict.values())
    return dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))

In [20]:
def decode_text(ru_corpus: str, encoded_text: str) -> str:
    corpus_freq_counter = get_ngrams(ru_corpus, 2) 
    encoded_text_freqs = get_ngrams(encoded_text, 2)
    decoder = dict(zip(encoded_text_freqs.keys(), corpus_freq_counter.keys()))
    decoded_text = ""
    i = 0
    while i <= len(encoded_text) - 2:
        decoded_text += decoder[encoded_text[i: i+2]]
        i += 2
    return decoded_text

In [21]:
decoded_example = decode_text(ru_corpus, encoded_example)

## Часть 2 - Результаты

In [22]:
len(example) == len(encoded_example) == len(decoded_example)

True

In [23]:
example

'оторые в первые времена христианства выходили возвещать добрую весть и так горячо верили что умирали за свою верутолстой с большим вниманием относился к этим идеям его герой константин левин тоже размышляет над тем что ему сказал брат николай разговор брата о коммунизме заставил его задуматься он считал переделку экономических условий вздором но он всегда чувствовал несправедливость своего избытка в сравнении с бедностью народа и впоследствии уже накануне  года беседуя с рабочими прохоровской ма'

In [24]:
encoded_example

'ызы щюпвпмю вщюпв юьюлхпш йдзйхлдзвхпвщшыяйъйпвыцвюсхзопяыа итпвюдзопйпзхнпеы кжыпвю йъйпжзыпиьй хъйпцхпдвытпвю изыъдзычпдпаыъогйьпвлйьхлйюьпызлыдйъдкпнпбзйьпйяюкьпюеыпею ычпнылдзхлзйлпъювйлпзыфюп хцьщгъкюзплхяпзюьпжзыпюьипднхцхъпа хзплйныъхчп хцеывы па хзхпыпныььилйцьюпцхдзхвйъпюеыпцхяиьхзодкпылпджйзхъпмю юяюънипбнылыьйжюднйшпидъывйчпвцяы ыьплыпылпвдюеяхпживдзвывхъплюдм хвюяъйвыдзопдвыюеыпйцащзнхпвпд хвлюлййпдпаюялыдзотплх ыяхпйпвмыдъюядзвййпифюплхнхлилюппеыяхпаюдюяикпдп хаыжйьйпм ышы ывднычпьх'

In [25]:
decoded_example

'аксты о оль азтомуов уа тртвтьнии жео амерл е  ослвылаорсев едо раалй лиинвастие снебы  ре пзан  н  таа до жо ь диски те вгоскховоо по епоовм пе бл обанвсть ибо я иот сееконоон р кзн ложсв ллиизто нойляичспна алиовре пк ся вде д чв ланашискокогрия  оужв лаа  сонексьс  зтадн кос чот с днн еалобм прочтыеноль какр хвсонрон ятилжизамилорыо щестт на су о х чиа епетеллоеннараиматка   ои ордоуля й ебькдео  в нль у э вгокарои ьннаажера е ез бмобрел эзаченаиннисьтоваера ких ннлеом негмен олкоемкоетонно е'

По-прежнему бессмысленно

# Часть 3 - MCMC-сэмплирование на биграммах

In [26]:
alphabet = list(ALPHABET)
unknown_ngram = 1 / len(ALPHABET) ** 2

In [27]:
def get_loglikelihood(text: str, corpus_freqs: Dict, ngram_len: int) -> float:
    ngram_counts = Counter([''.join(text[i: i + ngram_len]) for i in range(len(text) - ngram_len)])
    return np.sum([count * np.log(corpus_freqs.get(ngram, unknown_ngram))
                    for ngram, count in ngram_counts.items()])


def compare_loglikelihood(cur_loglh: float, new_loglh: float) -> bool:
    if new_loglh > cur_loglh:
        return True
    return np.random.rand() < np.exp(new_loglh - cur_loglh)


def decode(text: str, corpus_freqs: Dict, ngram_len: int, n_iter: int) -> str:
    best_version = copy(text)
    cur_loglh = best_loglh = get_loglikelihood(text, corpus_freqs, ngram_len)
    iter_num = 0
    for iteration in tqdm(range(n_iter)):
        iter_num += 1
        
        # меняю 2 рандомно выбранные буквы алфавита местами
        swapped = list(copy(text))
        letters = np.random.choice(alphabet, 2, replace=False)
        for i in range(len(swapped)):
            if swapped[i] == letters[0]:
                swapped[i] = letters[1]
            elif swapped[i] == letters[1]:
                swapped[i] = letters[0]
        swapped = ''.join(swapped)
        
        new_loglh = get_loglikelihood(swapped, corpus_freqs, ngram_len)
        if compare_loglikelihood(cur_loglh, new_loglh):
            text = swapped
            cur_loglh = new_loglh
            if cur_loglh > best_loglh:
                best_loglh = cur_loglh
                best_version = copy(text)
        
        if iter_num == 10 or iter_num % vebrocity == 0: # вывожу самое начало процесса и некоторые стадии
            print(f'Номер итерации: {iter_num}')
            print(''.join(best_version))

    return ''.join(best_version)

In [28]:
bigram_freqs = get_ngrams(ru_corpus, 2)

## Часть 3 - Результаты

In [29]:
vebrocity = 5000
decoded = decode(encoded_example, bigram_freqs, 2, 15000)

  5%|█▋                                   | 677/15000 [00:00<00:06, 2228.23it/s]

Номер итерации: 10
ызы щюпапмю ащюпа юьюлхпш йдзйхлдзахпащшыбйъйпаыцаюсхзопбыв итпаюдзопйпзхнпяы кжыпаю йъйпжзыпиьй хъйпцхпдаытпаю изыъдзычпдпвыъогйьпалйьхлйюьпызлыдйъдкпнпезйьпйбюкьпюяыпяю ычпнылдзхлзйлпъюайлпзыфюп хцьщгъкюзплхбпзюьпжзыпюьипднхцхъпв хзплйныъхчп хцяыаы пв хзхпыпныььилйцьюпцхдзхайъпюяыпцхбиьхзодкпылпджйзхъпмю юбюънипенылыьйжюднйшпидъыайчпацбы ыьплыпылпадюябхпжиадзаыахъплюдм хаюбъйаыдзопдаыюяыпйцвщзнхпапд халюлййпдпвюблыдзотплх ыбхпйпамыдъюбдзаййпифюплхнхлилюппяыбхпвюдюбикпдп хвыжйьйпм ышы ыаднычпьх


 36%|█████████████                       | 5450/15000 [00:02<00:04, 2272.91it/s]

Номер итерации: 5000
етерша в барвша вралано зристионство вшзедими вегвахоть депруы васть и ток жеряче варими чте улироми го свеы варутемстей с пемьъил внилониал етнесимся к этил идаял аже жарей кенстонтин мавин теёа роглшъмяат нод тал чте алу скогом прот никемой рогжевер прото е келлунигла гостовим аже годулоться ен считом барадамку экенеличаскиз усмевий вгдерел не ен всаждо чувствевом насбровадмивесть свеаже игпштко в сровнании с паднестьы норедо и вбесмадствии уёа ноконуна  жедо пасадуя с ропечили брезеревскей ло


 70%|████████████████████████▍          | 10449/15000 [00:04<00:02, 2271.65it/s]

Номер итерации: 10000
оторёе в первёе времена христианства вёходили возвеъать добруы весть и так горячо верили что умирали за своы верутолстой с большим вниманием относился к этим идеям его герой константин левин тоже размёшляет над тем что ему сказал брат николай разговор брата о коммунизме заставил его задуматься он считал переделку экономических условий вздором но он всегда чувствовал несправедливость своего избётка в сравнении с бедностьы народа и впоследствии уже накануне  года беседуя с рабочими прохоровской ма


100%|███████████████████████████████████| 15000/15000 [00:06<00:00, 2265.33it/s]

Номер итерации: 15000
оторые в первые времена христианства выходили возвешать добрую весть и так горячо верили что умирали за свою верутолстой с больъим вниманием относился к этим идеям его герой константин левин тоже размыъляет над тем что ему сказал брат николай разговор брата о коммунизме заставил его задуматься он считал переделку экономических условий вздором но он всегда чувствовал несправедливость своего избытка в сравнении с бедностью народа и впоследствии уже накануне  года беседуя с рабочими прохоровской ма


In [34]:
# По-моему круто получилось, тут для сравнения исходный кусочек текста 
example

'оторые в первые времена христианства выходили возвещать добрую весть и так горячо верили что умирали за свою верутолстой с большим вниманием относился к этим идеям его герой константин левин тоже размышляет над тем что ему сказал брат николай разговор брата о коммунизме заставил его задуматься он считал переделку экономических условий вздором но он всегда чувствовал несправедливость своего избытка в сравнении с бедностью народа и впоследствии уже накануне  года беседуя с рабочими прохоровской ма'

# Часть 4 - Расшифровка сообщения

In [30]:
text = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'


In [31]:
text_unigram_freqs = get_unigrams(text)
decoder = dict(zip(text_unigram_freqs, corpus_unigram_freqs))

preprocessed_text = ''.join([decoder[symbol] for symbol in text])

In [32]:
preprocessed_text

'олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю'

In [33]:
vebrocity = 10000
decoded = decode(preprocessed_text, bigram_freqs, 2, 40000)

  1%|▍                                    | 443/40000 [00:00<00:08, 4428.06it/s]

Номер итерации: 10
оина рд раяасо лекэтнулдь ана пемса лекэтнулдь совис б шсеые иеезжолач весекдь ноыве пкемастсу ивекоо риоые рд рио ияонтна пктрануле а пенбмасо этвиаэтнулдь зтнн йт пеиноялоо мосроксео йтятлао вбкит гесч веломле ч ламоые ло езожтю


 27%|█████████▎                         | 10617/40000 [00:02<00:06, 4402.18it/s]

Номер итерации: 10000
етни ся сичиве лорманыляь ини подви лорманыляь вектв у щвого тоошжелий коворяь негко продивавы ткорее стего ся сте тченани прасиныло и понудиве мактиманыляь шанн за потнечлее девсервое зачалие курта бовй коледло й лидего ле ошежац


 53%|██████████████████▋                | 21300/40000 [00:04<00:04, 4461.26it/s]

Номер итерации: 20000
етни сь симиве лорчаныльй ини подви лорчаныльй вектв у фвого тоожшелия коворьй негко продивавы ткорее стего сь сте тменани прасиныло и понудиве чактичаныльй жанн за потнемлее девсервое замалие курта бовя коледло я лидего ле ожешаэ


 77%|██████████████████████████▉        | 30720/40000 [00:06<00:02, 4468.98it/s]

Номер итерации: 30000
если вй вимите норчальнйы или подти норчальнйы текст у этого сообёения которйы легко продитать скорее всего вй все смелали правильно и полудите чаксичальнйы балл за послемнее детвертое замание курса хотя конедно я нидего не обеёаф


100%|███████████████████████████████████| 40000/40000 [00:09<00:00, 4434.01it/s]

Номер итерации: 40000
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю


Текст не удалось декодировать идельно, но по-моему его можно достаточно легко понять 

если вы виДите норМальный или почти норМальный текст у этого сооБЩения который легко прочитать скорее всего вы все сДелали правильно и получите МаксиМальный Балл за послеДнее четвертое заДание курса Хотя конечно я ничего не оБеЩаю

Часто перепутаны между собой М и Д (вимите, нордальный, смелали, даксидальный, послемнее, замание), а также встречается замена Б на Ж (жалл, ожещаю, соожщения)